In [1]:
from pyspark.sql import SparkSession, functions as f, types as t

In [ ]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

Дампи таблиць знаходяться в папці `data`.
Можете створювати стільки нових клітинок, скільки вам необхідно.

Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)

**Увага!** Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [30]:
from pyspark.sql import SparkSession, functions as f, types as t

In [31]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

In [32]:
film_category = spark.read.csv('home_task/data/film_category.csv', inferSchema=True, header=True)

In [33]:
film = spark.read.csv('home_task/data/film.csv', inferSchema=True, header=True)

In [34]:
category = spark.read.csv('home_task/data/category.csv', inferSchema=True, header=True)

In [35]:
joined_df = film_category.join(film, film_category.film_id == film.film_id, "inner") \
    .join(category, film_category.category_id == category.category_id) \
    .select(category.name, film.film_id)

In [36]:
group_sort_df = joined_df.groupBy("name").agg(f.sum("film_id").alias("sum_film_id")).orderBy(f.col("sum_film_id").desc())
group_sort_df.show(5)                                         

+-----------+-----------+
|       name|sum_film_id|
+-----------+-----------+
|     Sports|      39584|
|    Foreign|      35720|
|Documentary|      35461|
|      Drama|      32562|
|  Animation|      32427|
+-----------+-----------+
only showing top 5 rows



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [49]:
from pyspark.sql import SparkSession, functions as f, types as t

In [50]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

In [51]:
film_actor = spark.read.csv('home_task/data/film_actor.csv', inferSchema=True, header=True)

In [52]:
film = spark.read.csv('home_task/data/film.csv', inferSchema=True, header=True)

In [53]:
actor = spark.read.csv('home_task/data/actor.csv', inferSchema=True, header=True)

In [54]:
joined_df = actor.join(film_actor, film_actor.actor_id == actor.actor_id) \
    .join(film, film_actor.film_id == film.film_id) \
    .select(actor.first_name, actor.last_name)

In [56]:
group_sort_df = joined_df.groupBy("first_name", "last_name").count().orderBy(f.col("count").desc()).limit(10)
group_sort_df.show(20)                                         

+----------+-----------+-----+
|first_name|  last_name|count|
+----------+-----------+-----+
|     SUSAN|      DAVIS|   54|
|      GINA|  DEGENERES|   42|
|    WALTER|       TORN|   41|
|      MARY|     KEITEL|   40|
|   MATTHEW|     CARREY|   39|
|    SANDRA|     KILMER|   37|
|  SCARLETT|      DAMON|   36|
|    ANGELA|WITHERSPOON|   35|
|     HENRY|      BERRY|   35|
|    VIVIEN|   BASINGER|   35|
+----------+-----------+-----+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [69]:
from pyspark.sql import SparkSession, functions as f, types as t

In [70]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

In [73]:
inventory = spark.read.csv('home_task/data/inventory.csv', inferSchema=True, header=True)

In [74]:
film_category = spark.read.csv('home_task/data/film_category.csv', inferSchema=True, header=True)

In [76]:
joined_df = payment.join(rental, payment.customer_id == rental.customer_id) \
    .join(inventory, rental.inventory_id == inventory.inventory_id) \
    .join(film_category, inventory.film_id == film_category.film_id) \
    .join(category, film_category.category_id == category.category_id) \
    .select(category.name, payment.amount)

In [77]:
group_sort_df = joined_df.groupBy("name").agg(f.sum("amount").alias("sum_amount")).orderBy(f.col("sum_amount").desc()).limit(1)
group_sort_df.show(5)                                         

+------+------------------+
|  name|        sum_amount|
+------+------------------+
|Sports|138295.47000005541|
+------+------------------+



4.
Вивести назви фільмів, яких не має в inventory.
Запит має бути без оператора IN

In [97]:
from pyspark.sql import SparkSession, functions as f, types as t

In [98]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

In [99]:
inventory = spark.read.csv('home_task/data/inventory.csv', inferSchema=True, header=True)

In [100]:
film = spark.read.csv('home_task/data/film.csv', inferSchema=True, header=True)

In [101]:
joined_df = film.join(inventory, film.film_id == inventory.film_id, "left_outer").filter(f.col("inventory_id").isNull()).select(film.title)

In [102]:
group_sort_df = joined_df.orderBy(f.col("title"))
group_sort_df.show(5)                                         

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
+--------------------+
only showing top 5 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [17]:
from pyspark.sql import SparkSession, functions as f, types as t
from pyspark.sql.window import Window

In [18]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

In [19]:
actor = spark.read.csv('home_task/data/actor.csv', inferSchema=True, header=True)

In [20]:
film_actor = spark.read.csv('home_task/data/film_actor.csv', inferSchema=True, header=True)

In [21]:
film_category = spark.read.csv('home_task/data/film_category.csv', inferSchema=True, header=True)

In [22]:
category = spark.read.csv('home_task/data/category.csv', inferSchema=True, header=True)

In [23]:
joined_df = actor.join(film_actor, actor.actor_id == film_actor.actor_id, "left") \
    .join(film_category, film_category.film_id == film_actor.film_id, "left") \
    .join(category, category.category_id == film_category.category_id, "left") \
    .filter(category.name == "Children") \
    .select( actor.first_name, actor.last_name)

In [31]:
windowSpec = Window.orderBy(f.col("count").desc())
group_sort_df = joined_df.groupBy("first_name", "last_name").count() \
    .withColumn("row_number",f.row_number().over(windowSpec)) \
    .orderBy(f.col("count").desc()) \
    .filter(f.col("row_number") < 4)
group_sort_df.show(10)                                         

+----------+---------+-----+----------+
|first_name|last_name|count|row_number|
+----------+---------+-----+----------+
|     HELEN|   VOIGHT|    7|         1|
|     SUSAN|    DAVIS|    6|         2|
|     KEVIN|  GARLAND|    5|         3|
+----------+---------+-----+----------+



6.
Вивести міста з кількістю активних та неактивних клієнтів
(в активних customer.active = 1).
Результат відсортувати за кількістю неактивних клієнтів за спаданням.

In [4]:
from pyspark.sql import SparkSession, functions as f, types as t
from pyspark.sql.window import Window

In [5]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

In [6]:
customer = spark.read.csv('home_task/data/customer.csv', inferSchema=True, header=True)

In [7]:
address = spark.read.csv('home_task/data/address.csv', inferSchema=True, header=True)

In [8]:
city = spark.read.csv('home_task/data/city.csv', inferSchema=True, header=True)

In [14]:
unactive_df = customer.join(address, customer.address_id == address.address_id, "left") \
    .join(city, address.city_id == city.city_id, "left") \
    .filter(customer.active == 0) \
    .groupBy(city.city).count() \
    .withColumnRenamed("count", "unactive") \
    .withColumnRenamed("city", "city_unactive")

active_df = customer.join(address, customer.address_id == address.address_id, "left") \
    .join(city, address.city_id == city.city_id, "left") \
    .filter(customer.active == 1) \
    .groupBy(city.city).count() \
    .withColumnRenamed("count", "active") \
    .withColumnRenamed("city", "city_active")

full_df = unactive_df.join(active_df, unactive_df.city_unactive == active_df.city_active, "fullouter")
full_df_1 = full_df.fillna("").na.fill(value=0, subset=["unactive", "active"])
full_df_2 = full_df_1.select(f.concat("city_unactive", f.lit("_"), "city_active").alias("city"), "unactive", "active")
full_df_3 = full_df_2.orderBy(f.col("city_unactive").desc())
full_df_3.show()


+-------------------+--------+------+
|               city|unactive|active|
+-------------------+--------+------+
|          Xiangfan_|       1|     0|
|           Wroclaw_|       1|     0|
|          Uluberia_|       1|     0|
|       Szkesfehrvr_|       1|     0|
|   Southend-on-Sea_|       1|     0|
|         Pingxiang_|       1|     0|
|         Najafabad_|       1|     0|
|        Kumbakonam_|       1|     0|
|            Ktahya_|       1|     0|
|            Kamyin_|       1|     0|
|            Daxian_|       1|     0|
|     Coatzacoalcos_|       1|     0|
|  Charlotte Amalie_|       1|     0|
|           Bat Yam_|       1|     0|
|            Amroha_|       1|     0|
|_A Corua (La Corua)|       0|     1|
|              _Abha|       0|     1|
|         _Abu Dhabi|       0|     1|
|             _Adana|       0|     1|
|              _Acua|       0|     1|
+-------------------+--------+------+
only showing top 20 rows

